# Load libraries

In [9]:
%matplotlib notebook
import matplotlib.pyplot as plt
import numpy as np 
import pandas as pd
import sklearn
import imblearn
import pickle
from joblib import load, dump

from sklearn.metrics import f1_score, classification_report
from imblearn.ensemble import BalancedRandomForestClassifier

In [10]:
# Download features
# !scp -r -P 22334 -i ~/.ssh/id_rsa guanaco.inf.uach.cl:/home/shared/astro/PLAsTiCC/fats_featurs.tar.gz .
# !tar xzvf fats_features.tar.gz

In [11]:
seed = 0

# Get data

In [13]:
# Select seed, load ids and features

seed = 0
with open(f"ids/seed{seed}/maxClass15k/dataset_ids_before_balancing.pkl", "rb") as f:
    lc_ids = pickle.load(f)

features = {}
for subset in ['train', 'validation', 'test']:
    print(subset)
    tmp = []
    
#     tmp2 = lc_ids[subset][:10]
    
#     for lc_id in tmp2:
    for lc_id in lc_ids[subset]:
#         print(lc_id)
#         print("aoisdj")
#         lc_id = lc_id[:2]
        with open("features/fats"+str(int(lc_id))+".pkl", "rb") as f:
            tmp.append(load(f))
    features[subset] = pd.concat(tmp, axis=0)
    print(features[subset])

train
          CAR_sigma_0  CAR_mean_0  Meanvariance_0      Mean_0  \
oid                                                             
7351742     20.000000    2.266214        0.707708  262.791833   
5334772      0.000000    0.236859       36.020257   23.685873   
21499835     0.000000    0.031168        4.023204    3.116770   
6295051      2.556705   -1.751365       -0.902638  -10.408014   
15767115     2.523422   -0.502267       -2.538533   -4.025575   
...               ...         ...             ...         ...   
14331795    20.000000   -3.443465       -2.214465  -53.181430   
8328147     20.000000   -1.290566       -1.464549  -85.410769   
17247028     0.000000   -0.127248       -0.735633  -12.724842   
9153936      1.039496    0.007693       61.494964    0.221374   
1227691      0.000001   -0.116406       -2.452710  -11.640638   

          PercentAmplitude_0    Skew_0  AndersonDarling_0       Std_0  \
oid                                                                     
73

          CAR_sigma_0  CAR_mean_0  Meanvariance_0     Mean_0  \
oid                                                            
6175067      1.349721    0.126773        0.844876  16.922935   
4536915      0.000008    0.109222        0.851335  10.922165   
3547517      0.009883   -0.030253      -56.481320  -3.025302   
4761092      0.000000   -0.129309       -1.384531 -12.930887   
17620773     0.000000    0.262234        0.868464  26.223398   
...               ...         ...             ...        ...   
19322436     0.000000    0.071259        0.898988   7.125906   
3797822      7.458986   -0.884822       -0.674508 -83.687173   
7188334      0.000000    0.102717        2.402835  10.271712   
1863859      0.000012    0.093690        0.715385   9.368957   
8638336      0.000000   -0.042432       -2.296586  -4.243183   

          PercentAmplitude_0    Skew_0  AndersonDarling_0       Std_0  \
oid                                                                     
6175067             1

In [14]:
print(features["train"].shape)

(52360, 288)


In [ ]:
print(features["test"].fillna(-1000).values.shape)

print(lc_ids['labels_test'].astype('int').shape)

# Train RF model

In [ ]:
# Train balanced RF
# from sklearn.metrics import f1_score, classification_report
# from imblearn.ensemble import BalancedRandomForestClassifier

rf = BalancedRandomForestClassifier(n_estimators=500, criterion='entropy', replacement=True,
                                    max_depth=10, class_weight='balanced', n_jobs=8) 
                

rf.fit(
    features['train'].fillna(-1000).values, 
    lc_ids['labels_train'].astype('int'),
) 

# save RF trained model
dump(rf, '../../experiments/comparingModels/seed' + str(seed) + '/RF/trainedRF.joblib') 
print("RF model saved")

# preds = rf.predict(features['validation'].fillna(-1000).values)
# print(classification_report(preds, lc_ids['labels_validation'].astype('int')))
# # print(rf.features)          

# Load saved RF model

In [6]:
# # load file
rf = load('../../experiments/comparingModels/seed' + str(seed) + '/RF/trainedRF.joblib') 

# Get test model predictions

In [ ]:
# assert lc_ids["test"].shape == lc_ids["labels_test"].shape == preds.shape

In [ ]:
# Result on data set
trainPreds = rf.predict(features['train'].fillna(-1000).values)
validPreds = rf.predict(features['validation'].fillna(-1000).values)
testPreds = rf.predict(features['test'].fillna(-1000).values)

# print(trainPreds)
# print(validPreds)
# print(testPreds)

# save results
results = {
    
    # train
    "trainIds": lc_ids["train"],
    "trainLabels": lc_ids["labels_train"],
    "trainPredictions": trainPreds,
    
    # valid
    "validIds": lc_ids["validation"],
    "validLabels": lc_ids["labels_validation"],
    "validPredictions": validPreds,
    
    # test
    "testIds": lc_ids["test"],
    "testLabels": lc_ids["labels_test"],
    "testPredictions": testPreds,
}

# save object
a_file = open("../../experiments/comparingModels/seed" + str(seed) + "/RF/predictionsRF.pkl", "wb")
pickle.dump(results, a_file)
a_file.close()

# # print(preds)
# print(classification_report(preds, lc_ids['labels_test'][:10].astype('int')))

In [ ]:
# load files
# a_file = open("../../experiments/comparingModels/" + "/testPredictionsRF.pkl", "rb")
# output = pickle.load(a_file)
# print(output["testPredictions"])

In [ ]:
# # Features sorted by importance
# features['train'].columns.values[np.argsort(rf.feature_importances_)[::-1]]